# Test Badger Maps

In [1]:
!pip install pandas

You should consider upgrading via the '/home/andre/miniconda3/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from datetime import datetime

datos = pd.read_csv('Sample test file - Sheet1.csv')
pd.options.display.max_rows = 10
datos.head()

,First Name,Last Name,Street,Zip,City,Type,Last Check-In Date,Job,Phone,Company
0,Federico,García,Recogidas Street,18805,Granada,U,14/01/2018,Dentist,958 419182,Badger Maps
1,Ángel,Ganivet,Puerta Real Avenue,18805,Granada,U,06/02/2018,Salesman,917891232,Badger Maps
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,André,Citröen,Gran Via Road,18805,Granada,D,1/3/2018,Postman,971 659 123,Badger Maps
4,Anselmo,Ortega,King St,DE1 3LD,Almería,A,12/11/2017,Policeman,933 12 41 59,Wood SA


In [3]:
import sqlite3

conn = sqlite3.connect('badger.db')
conn.execute("DROP TABLE data")
conn.execute('''CREATE TABLE data
         (
         id INTEGER PRIMARY KEY,    
         first_name TEXT,
         second_name TEXT,     
         street           TEXT    NOT NULL,
         zip            TEXT     NOT NULL,
         city           TEXT     NOT NULL,
         type TEXT,
         job TEXT,
         phone TEXT,
         last_check_in_date DATETIME NOT NULL,
         company        TEXT     NOT NULL
         
         );''')


In [4]:
for index, row in datos.iterrows():
    if pd.isna(row['First Name']) and pd.isna(row['Last Name']) and pd.isna(row['Street']) and pd.isna(row['Zip']) and pd.isna(row['Type']) and pd.isna(row['City']) and pd.isna(row['Job']) and pd.isna(row['Phone']) and pd.isna(row['Street']) and pd.isna(row['Last Check-In Date']) and pd.isna(row['Company']):
        print('Fila ' + str(index)+ ' vacía')
        print(row['First Name'],row['Last Name'],row['Street'],row['Zip'],row['City'],row['Type'],row['Job'],row['Phone'],row['Last Check-In Date'],row['Company'],'\n') 
        continue
    if pd.isna(row['Street']) or pd.isna(row['Zip']) or pd.isna(row['Street']) or pd.isna(row['Last Check-In Date']) or pd.isna(row['Company']):
        print('A la fila '+ str(index) + ' Le falta un valor requerido')
        print(row['First Name'],row['Last Name'],row['Street'],row['Zip'],row['City'],row['Type'],row['Job'],row['Phone'],row['Last Check-In Date'],row['Company'],'\n')
        continue

    

    conn.execute("INSERT INTO data (first_name,second_name,street,zip,city,type,job,phone,last_check_in_date,company) VALUES (:fir,:sec,:str,:zip,:city,:type,:job,:pho,:last,:comp);",{
        'fir':row['First Name'],
        'sec':row['Last Name'],
        'str':row['Street'],
        'zip':row['Zip'],
        'city':row['City'],
        'type':row['Type'],
        'job':row['Job'],
        'pho':row['Phone'],
        'last':datetime.strptime(row['Last Check-In Date'], '%d/%m/%Y'),
        'comp':row['Company']
    }
        )
    

Fila 2 vacía
nan nan nan nan nan nan nan nan nan nan 

A la fila 6 Le falta un valor requerido
Markus Dupont 102, Fake Street 18005 Granada A Musician 912234223 nan Fabric SA 

A la fila 7 Le falta un valor requerido
Jake Robins Street Road nan Granada D Assistant 1111111111 11/03/2018 Fabric SA 

A la fila 8 Le falta un valor requerido
Lola Rivers nan 18005 Granada A CEO 958963229 21/02/2018 Fabric SA 



In [5]:
cur = conn.cursor()
cur.execute('SELECT * FROM data')
for row in cur.fetchall():
     print(row)

(1, 'Federico', 'García', 'Recogidas Street', '18805', 'Granada', 'U', 'Dentist', '958 419182', '2018-01-14 00:00:00', 'Badger Maps')
(2, 'Ángel', 'Ganivet', 'Puerta Real Avenue', '18805', 'Granada', 'U', 'Salesman', '917891232', '2018-02-06 00:00:00', 'Badger Maps')
(3, 'André', 'Citröen', 'Gran Via Road', '18805', 'Granada', 'D', 'Postman', '971 659 123', '2018-03-01 00:00:00', 'Badger Maps')
(4, 'Anselmo', 'Ortega', 'King St', 'DE1 3LD', 'Almería', 'A', 'Policeman', '933 12 41 59', '2017-11-12 00:00:00', 'Wood SA')
(5, 'Bjorn', 'Ostberg', '27, Queen St', 'DE1 3LD', 'Almería', 'U', 'Actor', '93 1 31 21 29', '2018-04-24 00:00:00', 'Wood SA')
(6, 'Paul', 'Hudson', 'Leicester Square', '18005', 'Granada', 'D', 'Salesman', '953912345', '2018-01-24 00:00:00', 'Fabric SA')


In [13]:
cur.execute("SELECT first_name,second_name,last_check_in_date as check_in FROM data ORDER BY last_check_in_date ASC")
for row in cur.fetchall():
     print(row)

('Anselmo', 'Ortega', '2017-11-12 00:00:00')
('Federico', 'García', '2018-01-14 00:00:00')
('Paul', 'Hudson', '2018-01-24 00:00:00')
('Ángel', 'Ganivet', '2018-02-06 00:00:00')
('André', 'Citröen', '2018-03-01 00:00:00')
('Bjorn', 'Ostberg', '2018-04-24 00:00:00')


In [12]:
cur.execute("SELECT first_name,second_name,last_check_in_date as check_in FROM data ORDER BY last_check_in_date DESC")
for row in cur.fetchall():
     print(row)

('Bjorn', 'Ostberg', '2018-04-24 00:00:00')
('André', 'Citröen', '2018-03-01 00:00:00')
('Ángel', 'Ganivet', '2018-02-06 00:00:00')
('Paul', 'Hudson', '2018-01-24 00:00:00')
('Federico', 'García', '2018-01-14 00:00:00')
('Anselmo', 'Ortega', '2017-11-12 00:00:00')


In [11]:
cur.execute("SELECT first_name,second_name as check_in FROM data ORDER BY first_name, second_name ASC")
for row in cur.fetchall():
     print(row)

('André', 'Citröen')
('Anselmo', 'Ortega')
('Bjorn', 'Ostberg')
('Federico', 'García')
('Paul', 'Hudson')
('Ángel', 'Ganivet')
